In [1]:
import cv2
import numpy as np
import math
import os
def preprocessing(pre_mag,pre_angle):#get magnitude and angle from RGB gradient images
    return_mag=np.zeros([128,64])
    return_angle=np.zeros([128,64])
    for i in range(128):
        for j in range(64):
            return_mag[i][j]=max(pre_mag[i][j])
            return_angle[i][j]=pre_angle[i][j][list(pre_mag[i][j]).index(return_mag[i][j])]
    return return_mag,return_angle

def buildHistogram(cell_mag,cell_angle):#build histogram for 8x8 cell
    histogram=np.zeros(9)
    for i in range(0,8):
        for j in range(0,8):
            if cell_angle[i][j]>180:
                cell_angle[i][j]=cell_angle[i][j]-180
            a=int(cell_angle[i][j]/20)
            if(a<8):
                histogram[a]=histogram[a]+float((cell_angle[i][j]-a*20)/20)*cell_mag[i][j]
                histogram[a+1]=histogram[a+1]+float((-cell_angle[i][j]+(a+1)*20)/20)*cell_mag[i][j]
            if(a==8):
                histogram[8]=histogram[8]+float((cell_angle[i][j]-160)/20)*cell_mag[i][j]
                histogram[0]=histogram[0]+float((180-cell_angle[i][j])/20)*cell_mag[i][j]
    return histogram

def block_Normalization(block_hist):#normalize 16x16 block return 36x1 vector
    sum=0
    for hist in block_hist:
        for i in hist:
            sum=sum+i*i
    sum=math.sqrt(sum)
    return_hist=[]
    for hist in block_hist:
        for i in hist:
            return_hist.append(float(i/sum))
    return return_hist

def histogramforpatch(patch):#build histogram for 128x64 patch
    patch = np.float32(patch) / 255.0
    patch = cv2.resize(patch, (64, 128))
    gx = cv2.Sobel(patch, cv2.CV_32F, 1, 0, ksize=1)
    gy = cv2.Sobel(patch, cv2.CV_32F, 0, 1, ksize=1)
    pre_mag, pre_angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    mag,angle=preprocessing(pre_mag,pre_angle)
    histograms = []
    h, w = patch.shape[:2]  # w=64,h=128
    for i in range(0, h - 8 + 1, 8):
        histogram_x = []
        for j in range(0, w - 8 + 1, 8):
            cell_mag = mag[i:i + 8, j:j + 8]
            cell_angle = angle[i:i + 8, j:j + 8]
            hist = buildHistogram(cell_mag, cell_angle)
            histogram_x.append(hist)       
        histograms.append(histogram_x)  
    # histograms (16,8,9)
    h_hist = len(histograms)
    w_hist = len(histograms[0])
    hist_normalization = []
    for i in range(h_hist - 1):
        for j in range(w_hist - 1):
            new_hist = block_Normalization(
                [histograms[i][j], histograms[i][j + 1], histograms[i + 1][j], histograms[i + 1][j + 1]])
            hist_normalization.append(new_hist)    
    return hist_normalization#(105,36)

def flatten(lists):#make hist_normalization (105,36) -> (105*36,1)
    return_list=[]
    for list in lists:
        for i in list:
            return_list.append(i)
    return return_list

def prepare():#get histogram and label for train and test
    histogram=[]
    label=[]   
    for object in objects:
        img_names=os.listdir('BOW/'+object)
        for img_name in img_names:
            img=cv2.imread('BOW/'+object+'/'+img_name)
            hist=flatten(histogramforpatch(img))
            if(not np.any(np.isnan(hist))):
                histogram.append(hist)
                label.append(objects.index(object))
    return histogram,label

objects=os.listdir('BOW')
histogram,label=prepare()
print('xong')

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


xong


In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
clf=svm.SVC(kernel='linear')
X_train,X_test,Y_train,Y_test=train_test_split(histogram,label,test_size=0.2,random_state = 0)
clf.fit(X_train,Y_train)
Y_predict=clf.predict(X_test)
print('Accuracy2 %.2f %%' %(100*accuracy_score(Y_test,Y_predict)))

Accuracy2 87.44 %


In [15]:
len(label)

1033

In [3]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score
clf = neighbors.KNeighborsClassifier(n_neighbors = 25)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

print("Accuracy1 %.2f %%" %(100*accuracy_score(Y_test, y_pred)))


Accuracy1 74.40 %


In [4]:
len(histogram[0])

3780

In [8]:
import pickle
with open('hog_feature.pkl','wb') as handle:
    pickle.dump((histogram,label),handle)